
# Datathon – EDA Avançado (Checklist) • v2
**Fonte de dados:** `../data/raw`  
Ajustes desta versão:
- Função `top_counts_clean` para gráficos **sem categoria "Vazio"**.
- **Remuneração** limitada a **R$ 50.000** no histograma.
- **Tempo de processamento** limitado a **200 dias**.
- **Bias & Fairness** com visualizações mais claras:
  - Funil **coeso** por estágio agregado (stacked bar normalizado).
  - **Heatmap** de proporções por sexo nos **Top N** status.
  - **Taxa de contratação** por sexo.


## 1) Setup e caminhos

In [ ]:

from pathlib import Path
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 160)

DATA_DIR = Path("../data/raw")
APPLICANTS = DATA_DIR / "applicants.json"
PROSPECTS  = DATA_DIR / "prospects.json"
VAGAS      = DATA_DIR / "vagas.json"

APPLICANTS, PROSPECTS, VAGAS


## 2) Carregamento, *flatten* e helpers

In [ ]:

def load_json(path: Path):
    if not path.exists():
        print(f"[AVISO] Não encontrado: {path.resolve()}")
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"[ERRO] Falha ao ler {path.name}: {e}")
        return None

def flatten_applicants(raw_dict: dict) -> pd.DataFrame:
    if not isinstance(raw_dict, dict):
        return pd.DataFrame()
    rows = []
    for codigo, dados in raw_dict.items():
        flat = {"codigo_profissional": str(codigo)}
        if isinstance(dados, dict):
            for bloco, conteudo in dados.items():
                if isinstance(conteudo, dict):
                    for k, v in conteudo.items():
                        flat[f"{bloco}.{k}"] = v
                else:
                    flat[bloco] = conteudo
        rows.append(flat)
    df = pd.DataFrame(rows)
    for col in ["infos_basicas.nome","informacoes_pessoais.nome"]:
        if col in df.columns:
            df["nome_candidato"] = df[col]
            break
    return df

def flatten_prospects(raw_dict: dict) -> pd.DataFrame:
    if not isinstance(raw_dict, dict):
        return pd.DataFrame()
    rows = []
    for vaga_id, vaga in raw_dict.items():
        titulo = None
        modalidade = None
        prospects = []
        if isinstance(vaga, dict):
            titulo = vaga.get("titulo")
            modalidade = vaga.get("modalidade")
            prospects = vaga.get("prospects", []) or []
        if not isinstance(prospects, list):
            continue
        for p in prospects:
            rec = dict(p)
            rec["vaga_id"] = str(vaga_id)
            rec["vaga_titulo"] = titulo
            rec["vaga_modalidade"] = modalidade
            if "codigo" in rec and pd.notna(rec["codigo"]):
                rec["codigo"] = str(rec["codigo"]).strip()
            rows.append(rec)
    df = pd.DataFrame(rows)
    if "situacao_candidado" in df.columns:
        df.rename(columns={"situacao_candidado": "situacao_candidato"}, inplace=True)
    for c in ["data_candidatura","ultima_atualizacao"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], format="%d-%m-%Y", errors="coerce")
    return df

def flatten_vagas(raw_dict: dict) -> pd.DataFrame:
    if not isinstance(raw_dict, dict):
        return pd.DataFrame()
    rows = []
    for vaga_id, dados in raw_dict.items():
        flat = {"vaga_id": str(vaga_id)}
        if isinstance(dados, dict):
            for bloco, conteudo in dados.items():
                if isinstance(conteudo, dict):
                    for k, v in conteudo.items():
                        flat[f"{bloco}.{k}"] = v
                else:
                    flat[bloco] = conteudo
        rows.append(flat)
    df = pd.DataFrame(rows)
    df.columns = [c.replace(" ", "_") for c in df.columns]
    for c in ["informacoes_basicas.data_requicisao","informacoes_basicas.limite_esperado_para_contratacao"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], format="%d-%m-%Y", errors="coerce")
    if "informacoes_basicas.titulo_vaga" in df.columns:
        df["titulo_vaga"] = df["informacoes_basicas.titulo_vaga"]
    if "perfil_vaga.estado" in df.columns:
        df["estado"] = df["perfil_vaga.estado"]
    if "perfil_vaga.cidade" in df.columns:
        df["cidade"] = df["perfil_vaga.cidade"]
    if "informacoes_basicas.tipo_contratacao" in df.columns:
        df["tipo_contratacao"] = df["informacoes_basicas.tipo_contratacao"]
    if "informacoes_basicas.analista_responsavel" in df.columns:
        df["analista_responsavel"] = df["informacoes_basicas.analista_responsavel"]
    return df

import numpy as np
def top_counts_clean(df, col, top=20, title=None):
    if df.empty or col not in df.columns:
        print(f"[SKIP] {col}")
        return
    series = df[col].astype(str).str.strip().replace(
        ["", "nan", "NA", "None", "N/A", "vazio", "Vazio"], np.nan
    ).dropna()
    vc = series.value_counts().head(top)
    ax = vc.plot(kind="barh", figsize=(10,4))
    ax.set_title(title or f"Top {top} – {col}")
    ax.set_xlabel("contagem")
    ax.grid(True, axis="x")
    plt.show()


## 3) Carregar DataFrames e Joins

In [ ]:

raw_app = load_json(APPLICANTS)
raw_pro  = load_json(PROSPECTS)
raw_vag  = load_json(VAGAS)

df_app = flatten_applicants(raw_app) if raw_app is not None else pd.DataFrame()
df_pro = flatten_prospects(raw_pro) if raw_pro is not None else pd.DataFrame()
df_vag = flatten_vagas(raw_vag) if raw_vag is not None else pd.DataFrame()

print("Applicants:", df_app.shape)
print("Prospects :", df_pro.shape)
print("Vagas     :", df_vag.shape)

df_join_pa = pd.DataFrame()
df_full = pd.DataFrame()
if not df_pro.empty and not df_app.empty:
    df_app["codigo_profissional"] = df_app["codigo_profissional"].astype(str)
    df_pro["codigo"] = df_pro["codigo"].astype(str)
    df_join_pa = df_pro.merge(df_app, left_on="codigo", right_on="codigo_profissional", how="left", suffixes=("_pro","_app"))
if not df_join_pa.empty and not df_vag.empty:
    df_full = df_join_pa.merge(df_vag, on="vaga_id", how="left", suffixes=("","_vaga"))

df_app.head(2), df_pro.head(2), df_vag.head(2)


## 4) Qualidade dos dados & distribuições limpas

In [ ]:

# % preenchimento (Applicants)
if not df_app.empty:
    display((df_app.notna().mean().sort_values(ascending=False) * 100).to_frame("pct_preenchimento").head(30))

# Distribuições (limpas)
top_counts_clean(df_app, "formacao_e_idiomas.nivel_academico", 20, "Nível acadêmico (Applicants)")
top_counts_clean(df_vag, "estado", 20, "Estados com mais vagas")
top_counts_clean(df_vag, "tipo_contratacao", 10, "Tipos de contratação")
top_counts_clean(df_pro, "recrutador", 20, "Recrutadores – volume de prospects")


## 5) Funil e conversão

In [ ]:

if not df_pro.empty and "situacao_candidato" in df_pro.columns:
    top_counts_clean(df_pro, "situacao_candidato", 20, "Distribuição – situacao_candidato")
    conv_recrutador = (df_pro.assign(is_contratado=df_pro["situacao_candidato"].fillna("").str.lower().str.contains("contratado"))
                       .groupby("recrutador")["is_contratado"].mean()
                       .sort_values(ascending=False))
    ax = conv_recrutador.head(10).plot(kind="barh", figsize=(8,4))
    ax.set_title("Taxa de contratação por recrutador (Top 10)")
    ax.grid(True, axis="x")
    plt.show()


## 6) Remuneração declarada (cap em R$ 50.000)

In [ ]:

col = "informacoes_profissionais.remuneracao"
if not df_app.empty and col in df_app.columns:
    remun = (df_app[col].astype(str)
             .str.replace("[^0-9,]", "", regex=True)
             .str.replace(",", ".", regex=False))
    with np.errstate(all="ignore"):
        remun = pd.to_numeric(remun, errors="coerce")
    # Cap and floor
    remun = remun.clip(lower=0, upper=50000)
    ax = remun.plot(kind="hist", bins=40, figsize=(10,4))
    ax.set_title("Distribuição – Remuneração declarada (cap 50k)")
    ax.set_xlabel("R$")
    ax.set_xlim(0, 50000)
    ax.grid(True, axis="y")
    plt.show()


## 7) Tempo de processamento (cap em 200 dias)

In [ ]:

if not df_pro.empty and {"data_candidatura","ultima_atualizacao"}.issubset(df_pro.columns):
    df_pro["tempo_processamento"] = (df_pro["ultima_atualizacao"] - df_pro["data_candidatura"]).dt.days
    tp = df_pro["tempo_processamento"].clip(lower=0, upper=200).dropna()
    ax = tp.plot(kind="hist", bins=40, figsize=(10,4))
    ax.set_title("Tempo de processamento (cap 200 dias)")
    ax.set_xlabel("dias")
    ax.set_xlim(0, 200)
    ax.grid(True, axis="y")
    plt.show()


## 8) Bias & Fairness – visualizações mais claras

In [ ]:

# 8.1 Mapeia status detalhados para estágios agregados de funil
def map_stage(status: str) -> str:
    if not isinstance(status, str): return "Desconhecido"
    s = status.lower()
    if "inscrito" in s or "prospect" in s:
        return "Inscrito/Prospect"
    if "encaminh" in s or "em avaliação" in s:
        return "Triagem/Encaminhado"
    if "entrevista" in s:
        return "Entrevista"
    if "proposta" in s:
        return "Proposta"
    if "documenta" in s:
        return "Documentação"
    if "contrat" in s:
        return "Contratado"
    if "não aprovado" in s or "nao aprovado" in s or "reprov" in s or "recus" in s or "desist" in s or "sem interesse" in s:
        return "Recusa/Reprovação"
    return "Outros"

viz = pd.DataFrame()
if not df_join_pa.empty and {"informacoes_pessoais.sexo","situacao_candidato"}.issubset(df_join_pa.columns):
    tmp = df_join_pa[["informacoes_pessoais.sexo","situacao_candidato"]].copy()
    # limpa sexo
    tmp["sexo"] = tmp["informacoes_pessoais.sexo"].astype(str).str.strip().replace(
        ["", "nan", "NA", "None", "N/A", "vazio", "Vazio"], np.nan
    )
    tmp = tmp.dropna(subset=["sexo", "situacao_candidato"])
    tmp["estagio"] = tmp["situacao_candidato"].apply(map_stage)

    # 8.1 Stacked bar normalizado por sexo (estágios agregados)
    stage_tab = pd.crosstab(tmp["sexo"], tmp["estagio"]).pipe(lambda t: t.div(t.sum(axis=1), axis=0))
    ax = stage_tab.plot(kind="bar", stacked=True, figsize=(10,6))
    ax.set_title("Distribuição por estágio do funil (normalizado) • por sexo")
    ax.set_ylabel("proporção")
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.tight_layout()
    plt.show()

    # 8.2 Heatmap de proporções por sexo nos Top N status detalhados
    N = 8
    top_status = (tmp["situacao_candidato"].value_counts().head(N).index.tolist())
    heat = tmp[tmp["situacao_candidato"].isin(top_status)]
    heat_tab = pd.crosstab(heat["sexo"], heat["situacao_candidato"]).pipe(lambda t: t.div(t.sum(axis=1), axis=0))

    fig, ax = plt.subplots(figsize=(1.5 * N, 4))
    im = ax.imshow(heat_tab.values, aspect="auto")
    ax.set_xticks(range(len(heat_tab.columns))); ax.set_xticklabels(heat_tab.columns, rotation=45, ha="right")
    ax.set_yticks(range(len(heat_tab.index))); ax.set_yticklabels(heat_tab.index)
    ax.set_title("Heatmap: proporção por sexo nos Top status")
    for i in range(heat_tab.shape[0]):
        for j in range(heat_tab.shape[1]):
            ax.text(j, i, f"{heat_tab.values[i,j]:.0%}", ha="center", va="center", fontsize=9, color="white" if heat_tab.values[i,j]>0.5 else "black")
    fig.colorbar(im, ax=ax, fraction=0.02, pad=0.04)
    plt.tight_layout()
    plt.show()

    # 8.3 Taxa de contratação por sexo
    hired = tmp.assign(is_hired=tmp["situacao_candidato"].str.lower().str.contains("contratado"))
    rate = hired.groupby("sexo")["is_hired"].mean().sort_values(ascending=False)
    ax = rate.plot(kind="bar", figsize=(6,4))
    ax.set_title("Taxa de contratação por sexo")
    ax.set_ylabel("proporção")
    ax.grid(True, axis="y")
    plt.show()
else:
    print("[INFO] df_join_pa vazio ou colunas ausentes para visualizações de fairness.")


## 9) Matching exploratório por keywords

In [ ]:

TECHS = ["python","java","sql","aws","docker","kubernetes","excel","sap","power_bi","sql_server","spark"]

def overlap_score(a, b, keywords):
    if not isinstance(a, str) or not isinstance(b, str):
        return 0.0
    A = {kw for kw in keywords if re.search(rf"\b{re.escape(kw)}\b", a.lower())}
    B = {kw for kw in keywords if re.search(rf"\b{re.escape(kw)}\b", b.lower())}
    return len(A & B) / max(1, len(A | B))

if 'df_full' in globals() and not df_full.empty:
    cand_cols = [c for c in ["cv_pt","informacoes_profissionais.conhecimentos_tecnicos"] if c in df_full.columns]
    vaga_cols = [c for c in ["perfil_vaga.principais_atividades","perfil_vaga.competencia_tecnicas_e_comportamentais","titulo_vaga"] if c in df_full.columns]
    if cand_cols and vaga_cols:
        df_full["_cand_text"] = df_full[cand_cols].agg(lambda x: " ".join(x.astype(str)), axis=1)
        df_full["_vaga_text"] = df_full[vaga_cols].agg(lambda x: " ".join(x.astype(str)), axis=1)
        df_full["match_score"] = df_full.apply(lambda r: overlap_score(r["_cand_text"], r["_vaga_text"], TECHS), axis=1)
        ax = df_full["match_score"].plot(kind="hist", bins=20, figsize=(8,4))
        ax.set_title("Distribuição de match_score (keywords)")
        ax.grid(True, axis="y")
        plt.show()
        display(df_full[["vaga_id","codigo","match_score"]].head(10))
